In [1]:
import pandas as pd
import spacy 
nlp = spacy.load('en')

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter 
from bs4 import BeautifulSoup 
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
stop = set(stopwords.words('english'))
import string

import pickle

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

### Load Data

In [4]:
# LOADS CSV WITH SELECTED DATA
df = pd.read_csv('conversation_tagged.csv')
conversations = df['Sound Bite Text']

#### Function to Clean Text used later

In [5]:
# CLEANS TEXT BY REPLACING CHARACTERS
def getCleanText(texts, lower = True):
    theList=[]
    for text in texts:
        text = str(text)
        parse_text = BeautifulSoup(text).get_text()
        letters_only = re.sub(r'http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', 
                              parse_text, flags=re.MULTILINE)
        letters_only = re.sub('^(1?[0-9]|2[0-3]):[0-5][0-9]$','time',letters_only)
        letters_only = re.sub('(name|NAME)\d{1,}','username',letters_only)
        letters_only = re.sub("[^a-zA-Z0-9\.:']",  
                          " ",                   
                          letters_only)
        letters_only = letters_only.replace('\n'," ")
        letters_only = letters_only.replace('\r'," ")
        letters_only = re.sub('[.]{2,}', '. ', letters_only)
        if lower:
            letters_only = letters_only.lower()
        words = CountVectorizer(stop_words='english').build_tokenizer()(letters_only)
        meaningful_words = [ w for w in words if len(w)> 0 and len(w)<20]
        clean_text = " ".join(meaningful_words)
        theList.append(clean_text)
    finalList = "_".join(theList)
    return finalList

In [6]:
#define some parameters  
noisy_pos_tags = ['PROP'] # TAGGING OF PROP
min_token_length = 2 # TWO CHARACTERS OR TWO TOKENS?

#Function to check if the token is a noise or not  
def isNoise(token):     
    is_noise = False
    if token.pos_ in noisy_pos_tags: # FROM LIST ABOVE
        is_noise = True 
    elif token.is_stop == True: # FROM STOP WORDS LIST 
        is_noise = True
    elif len(token.string) <= min_token_length: # FROM PARAMETER ABOVE
        is_noise = True
    return is_noise 
def cleanup(token, lower = True):
    if lower: # TURNS ALL TEXT TO LOWERCASE
        token = token.lower()
    return token.strip() # REMOVES SPACES FROM THE BEGINNING AND END

### Named Entity Recognition

In [7]:
def get_entities(sent):
    try:
        document = nlp(sent)
    except:
        return ["error"]
    
    entities_list = []
    labels = set([w.label_ for w in document.ents]) 
    for label in labels: 
        entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
        entities = list(set(entities))
        if label == "ORG": # PICKS ORG ENTITIES
            entities_list.extend(entities)
    if len(entities_list) > 0:
        return entities_list
    else:
        return ["<UNK>"]
org_list = map(get_entities, conversations)
org_list = list(org_list)

#### Concatentate Named Entities together from all rows

In [8]:
all_orgs_clean = map(getCleanText, org_list)
entities_per_text = list(all_orgs_clean)

/home/k/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/k/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


#### Count of Named Entitites

In [9]:
entities_dict = Counter(entities_per_text)
entities_list_sorted = sorted(entities_dict, key = entities_dict.get, reverse=True)
entities_list_sorted_tup = [(entity,entities_dict[entity]) for entity in entities_list_sorted]
#entities_list_sorted_tup

#### Add list as column and Check for consitency

In [10]:
df = df.assign(entities = entities_per_text)
print(df.shape)
print(len(entities_per_text))

(18340, 17)
18340


In [11]:
print(entities_per_text[:5])

['pepsico inc_elise amendola_pepsi_3q_file ap photo_pepsico inc_the associated press_mobile apps file_file ap', 'pepsico inc_elise amendola_pepsi_3q_file ap photo_pepsico inc_the associated press_mobile apps file_file ap', 'the district of columbia_chrysler_mcdonald_coca cola_executive creative director ashley sword_stellar novellas_stellar scholars_roundtable_stellar romance', 'crisps_coca cola_walkers', 'coca cola_instagram']


#### Dump files

In [12]:
dump(df, "../dumps/df_sel_entities.pkl")
dump(entities_list_sorted_tup, "../dumps/df__sel_entities_count.pkl")
dump(org_list, "../dumps/df_sel_entities_list.pkl")